# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [70]:
import pandas as pd
import os
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re
import pickle

nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [22]:
# load data from database
engine = create_engine('sqlite:///DisasterMessage.db')
df = pd.read_sql_table('DisasterMessage', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
X = df['message']
Y = df[df.columns.to_list()[4:]]

print(X.shape)
print(Y.shape)

(26216,)
(26216, 36)


### 2. Write a tokenization function to process your text data

In [24]:
def tokenize(text: str) -> list[str]:
    stop_words = stopwords.words("english")

    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # tokenize text
    tokens = word_tokenize(text.lower())

    # lemmatize and remove stop words
    tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens if w not in stop_words]

    return tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [32]:
pipeline = Pipeline([
    ('countv', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [33]:
X_train, X_test, Y_train, Y_test  = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('countv',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000019C05931BC0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
Y_predict = pipeline.predict(X_test)


In [47]:
cols = df.columns[4:].to_list()

for i, col in enumerate(df.columns[4:].to_list()):
    print('category:', col)
    print(classification_report(Y_test[col].to_numpy(), Y_predict[:, i]))


category: related
              precision    recall  f1-score   support

           0       0.68      0.44      0.53      1543
           1       0.84      0.93      0.88      4959
           2       0.46      0.35      0.40        52

    accuracy                           0.81      6554
   macro avg       0.66      0.57      0.60      6554
weighted avg       0.80      0.81      0.80      6554

category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5448
           1       0.86      0.49      0.62      1106

    accuracy                           0.90      6554
   macro avg       0.88      0.73      0.78      6554
weighted avg       0.90      0.90      0.89      6554

category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       0.5

c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6400
           1       0.75      0.02      0.04       154

    accuracy                           0.98      6554
   macro avg       0.86      0.51      0.51      6554
weighted avg       0.97      0.98      0.97      6554

category: missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6484
           1       0.00      0.00      0.00        70

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

category: refugees
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6369
           1       0.75      0.02      0.03       185

    accuracy                           0.97      6554
   macro avg       0.86      0.51      0.51      6554
weighted avg       0.97      0

c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

In [ ]:
# print(Y_test[Y_test['related']==2])

### 6. Improve your model
Use grid search to find better parameters. 

In [55]:
pipeline.get_params()

{'memory': None,
 'steps': [('countv',
   CountVectorizer(tokenizer=<function tokenize at 0x0000019C05931BC0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'countv': CountVectorizer(tokenizer=<function tokenize at 0x0000019C05931BC0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'countv__analyzer': 'word',
 'countv__binary': False,
 'countv__decode_error': 'strict',
 'countv__dtype': numpy.int64,
 'countv__encoding': 'utf-8',
 'countv__input': 'content',
 'countv__lowercase': True,
 'countv__max_df': 1.0,
 'countv__max_features': None,
 'countv__min_df': 1,
 'countv__ngram_range': (1, 1),
 'countv__preprocessor': None,
 'countv__stop_words': None,
 'countv__strip_accents': None,
 'countv__token_pattern': '(?u)\\b\\w\\w+\\b',
 'countv__tokenizer': <function __main__.tokenize(text: str) -> list[str]>,
 'countv__vocabulary': None,
 'tfidf__norm': 'l2',
 't

In [62]:
parameters = {
    'tfidf__norm': ['l2', 'l1'],
    # 'tfidf__use_idf': [True, False],
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters)

In [63]:
cv.fit(X_train, Y_train)

c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 't

GridSearchCV(estimator=Pipeline(steps=[('countv',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000019C05931BC0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'tfidf__norm': ['l2', 'l1']})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [64]:
Y_predict = cv.predict(X_test)

In [65]:
cols = df.columns[4:].to_list()

for i, col in enumerate(df.columns[4:].to_list()):
    print('category:', col)
    print(classification_report(Y_test[col].to_numpy(), Y_predict[:, i]))

category: related
              precision    recall  f1-score   support

           0       0.68      0.43      0.53      1543
           1       0.84      0.94      0.88      4959
           2       0.42      0.31      0.36        52

    accuracy                           0.81      6554
   macro avg       0.65      0.56      0.59      6554
weighted avg       0.80      0.81      0.80      6554

category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5448
           1       0.85      0.49      0.62      1106

    accuracy                           0.90      6554
   macro avg       0.88      0.74      0.78      6554
weighted avg       0.90      0.90      0.89      6554

category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       0.5

c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6062
           1       0.64      0.07      0.13       492

    accuracy                           0.93      6554
   macro avg       0.79      0.53      0.55      6554
weighted avg       0.91      0.93      0.90      6554

category: medical_products
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6252
           1       0.87      0.09      0.16       302

    accuracy                           0.96      6554
   macro avg       0.91      0.54      0.57      6554
weighted avg       0.95      0.96      0.94      6554

category: search_and_rescue
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6381
           1       0.62      0.05      0.09       173

    accuracy                           0.97      6554
   macro avg       0.80      0.52      0.54      6554
weighted avg       

c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\linhh\anaconda3\envs\udacity-env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        43

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

category: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6493
           1       0.00      0.00      0.00        61

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

category: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [71]:
filename = "classifier.pkl"
# pickle.dump(cv, open(filename, 'wb'))
pickle.dump(cv, open(os.path.join(os.getcwd(), filename), 'wb'))

In [74]:
loaded_model = pickle.load(open(os.path.join(os.getcwd(), filename), 'rb'))
loaded_model.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('countv',
   CountVectorizer(tokenizer=<function tokenize at 0x0000019C05931BC0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'estimator__verbose': False,
 'estimator__countv': CountVectorizer(tokenizer=<function tokenize at 0x0000019C05931BC0>),
 'estimator__tfidf': TfidfTransformer(),
 'estimator__clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'estimator__countv__analyzer': 'word',
 'estimator__countv__binary': False,
 'estimator__countv__decode_error': 'strict',
 'estimator__countv__dtype': numpy.int64,
 'estimator__countv__encoding': 'utf-8',
 'estimator__countv__input': 'content',
 'estimator__countv__lowercase': True,
 'estimator__countv__max_df': 1.0,
 'estimator__countv__max_features': None,
 'estimator__countv__min_df': 1,
 'estimator__countv__ngram_range': (1, 1),
 'estimator__countv__preprocessor': None,
 'esti

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.